# Creat a model with tensorflow model maker

In [5]:
#install all required packages

!sudo apt -y install libportaudio2
!pip install -q --use-deprecated=legacy-resolver tflite-model-maker
!pip install -q pycocotools
!pip install -q opencv-python-headless==4.1.2.30
!pip uninstall -y tensorflow && pip install -q tensorflow==2.8.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 19 not upgraded.
Need to get 64.6 kB of archives.
After this operation, 215 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Fetched 64.6 kB in 0s (171 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype


In [ ]:
#Import all alibrary and model configurations  
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

## Prepare the dataset
Here we need **PASCAL VOC** formate for training model 
you can label dataset by RoboFlow website
**train_data** for training and **validation** for evaluate model

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install unzip
!unzip "/content/drive/MyDrive/data2/Crossing_road.v1i.voc.zip"
# !unzip "/content/valid2.zip"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for unzip: filename=unzip-1.0.0-py3-none-any.whl size=1319 sha256=6bf0101156417f6b606a5d6f998649958a49996e543fa9695160bcd7ee5e9003
  Stored in directory: /root/.cache/pip/wheels/c5/05/b3/f7b36dbaaf76de31b718cde792c953bfd11d2414a72f204b56
Successfully built unzip
Archive:  /content/drive/MyDrive/data2/Crossing_road.v1i.voc.zip
  inflating: Crossing_road.v1i.voc/README.dataset.txt  
  inflating: Crossing_road.v1i.voc/README.roboflow.txt  
   creating: Crossing_road.v1i.voc/test/
  inflating: Crossing_road.v1i.voc/test/0064880cfb7b82ac_jpg.rf.f723bba9772620bd4e844aad823fa18b.jpg  
  inflating: Crossing_road.v1i.voc/test/0064880cfb7b82ac_jpg.rf.f723bba9772620bd4e844aad823fa18b.xml  
  inflating: Crossing_road.v1i.voc/test/050adeedad9159b6_jpg.rf.72a39dd567ff1aa3731cfc24767c8cec.jpg  
  inflating: Crossing_road.v1i.voc/test/050adeedad9159b6_jpg.rf.72a39dd567ff1aa3731cfc24767c8c

In [5]:
#define train, vali and test datasets
train_data = object_detector.DataLoader.from_pascal_voc('/content/Crossing_road.v1i.voc/train','/content/Crossing_road.v1i.voc/train',['Traffic-light','crossing-line'])
validation_data = object_detector.DataLoader.from_pascal_voc('/content/Crossing_road.v1i.voc/valid','/content/Crossing_road.v1i.voc/valid',['Traffic-light','crossing-line'])
test_data = object_detector.DataLoader.from_pascal_voc('/content/Crossing_road.v1i.voc/test','/content/Crossing_road.v1i.voc/test',['Traffic-light','crossing-line'])

INFO:tensorflow:Cache will be stored in /tmp/tmp17dii1ml with prefix filename 7bee435cfee6949ada7cad38d32e815e. Cache_prefix is /tmp/tmp17dii1ml/7bee435cfee6949ada7cad38d32e815e
INFO:tensorflow:On image 0
INFO:tensorflow:On image 100
INFO:tensorflow:On image 200
INFO:tensorflow:On image 300
INFO:tensorflow:On image 400
INFO:tensorflow:On image 500
INFO:tensorflow:On image 600
INFO:tensorflow:On image 700
INFO:tensorflow:On image 800
INFO:tensorflow:Cache will be stored in /tmp/tmpturtwg27 with prefix filename 11a526a320c4c8a1416196ba3144593c. Cache_prefix is /tmp/tmpturtwg27/11a526a320c4c8a1416196ba3144593c
INFO:tensorflow:On image 0
INFO:tensorflow:Cache will be stored in /tmp/tmpnylyes34 with prefix filename fd42585a6188df7d3abc4e516ad0ebb5. Cache_prefix is /tmp/tmpnylyes34/fd42585a6188df7d3abc4e516ad0ebb5
INFO:tensorflow:On image 0


# choose and download Model
* In this step we choose which model is suitable for our task and we can see the accuracy and Latency in blow table

Here is the performance of each EfficientDet-Lite models compared to each others.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 37            | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 49            | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 69            | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 116           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 260           | 41.96%               |

<i> * Size of the integer quantized models. <br/>
** Latency measured on Pixel 4 using 4 threads on CPU. <br/>
*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.
</i>

# For tuning model 
- Epochs: it is integer and **50** by default. the high number can reach over accuracy 
- Batch_size: it is Integer and 64 by defualt.The number of samples to use in one training step. Having good *GPU* can choose higher number for better accuracy.
- Train_whole_model: Boolen, False by defualt. If true, train the whole model. Otherwise, only train the layers that do not match var_freeze_expr.

In [6]:
#chossing the Model Architecture 
spec = model_spec.get('efficientdet_lite2') 

In [7]:
# Train model with our data set
# model_dir: the location to save model checkpoint file 
#steps_per_execution:  Number of steps per training execution
#tflite_max_detections:  integer, 25 by default. The max number of output detections in the TFLite model.
# Strategy:  A string specifying which distribution strategy to use. Accepted values are 'tpu', 'gpus', None. tpu' means to use TPUStrategy. 'gpus' mean to use MirroredStrategy for multi-gpus. If None, use TF default with OneDeviceStrategy.


model = object_detector.create(train_data,model_spec=spec,batch_size=16,epochs=50, train_whole_model=True, validation_data=validation_data)

INFO:tensorflow:Retraining the models...
Epoch 1/50
51/51 [==============================] - 103s 950ms/step - det_loss: 1.6387 - cls_loss: 0.9210 - box_loss: 0.0144 - reg_l2_loss: 0.0759 - loss: 1.7146 - learning_rate: 0.0140 - gradient_norm: 2.1040 - val_det_loss: 1.3824 - val_cls_loss: 0.8497 - val_box_loss: 0.0107 - val_reg_l2_loss: 0.0760 - val_loss: 1.4584
Epoch 2/50
51/51 [==============================] - 46s 902ms/step - det_loss: 1.0049 - cls_loss: 0.5783 - box_loss: 0.0085 - reg_l2_loss: 0.0762 - loss: 1.0811 - learning_rate: 0.0200 - gradient_norm: 4.3279 - val_det_loss: 1.4735 - val_cls_loss: 0.9237 - val_box_loss: 0.0110 - val_reg_l2_loss: 0.0766 - val_loss: 1.5500
Epoch 3/50
51/51 [==============================] - 46s 903ms/step - det_loss: 0.8599 - cls_loss: 0.4931 - box_loss: 0.0073 - reg_l2_loss: 0.0768 - loss: 0.9368 - learning_rate: 0.0199 - gradient_norm: 3.5489 - val_det_loss: 0.8839 - val_cls_loss: 0.5169 - val_box_loss: 0.0073 - val_reg_l2_loss: 0.0771 - val_lo

# Evaluate model
# Convert Model to different Format
- ExportFormat.TFLITE
- ExportFormat.LABEL
- ExportFormat.SAVED_MODEL


In [9]:
model.evaluate(validation_data)

2/2 [==============================] - 16s 5s/step

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
Converting ndarray to lists...
(8900, 7)
0/8900
DONE (t=0.04s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.77s).
Accumulating evaluation results...
DONE (t=0.09s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.418
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.673
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.456
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.502
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.535
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.472
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.345
 Average Recall     (AR) @[ IoU=0.50:0.95 | a

{'AP': 0.41799912,
 'AP50': 0.6727533,
 'AP75': 0.45569366,
 'AP_/Traffic-light': 0.18305936,
 'AP_/crossing-line': 0.65293884,
 'APl': 0.47197583,
 'APm': 0.53519046,
 'APs': 0.502235,
 'ARl': 0.6513889,
 'ARm': 0.6548529,
 'ARmax1': 0.3451271,
 'ARmax10': 0.5068046,
 'ARmax100': 0.5322034,
 'ARs': 0.5568653}

In [10]:
model.export(export_dir='.', export_format=[ExportFormat.SAVED_MODEL, ExportFormat.LABEL])


INFO:tensorflow:Assets written to: ./saved_model/assets


INFO:tensorflow:Assets written to: ./saved_model/assets


INFO:tensorflow:Saving labels in ./labels.txt.


INFO:tensorflow:Saving labels in ./labels.txt.


In [11]:
model.export(export_dir='.',tflite_filename='model_maker.tflite')

INFO:tensorflow:Assets written to: /tmp/tmpogte_f9z/assets


INFO:tensorflow:Assets written to: /tmp/tmpogte_f9z/assets


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in /tmp/tmpb5j3l7hv/labelmap.txt.


INFO:tensorflow:Saving labels in /tmp/tmpb5j3l7hv/labelmap.txt.


TypeError: ignored

In [12]:
model.evaluate_tflite('model_maker.tflite', validation_data)

89/89 [==============================] - 696s 8s/step

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
Converting ndarray to lists...
(2225, 7)
0/2225
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.24s).
Accumulating evaluation results...
DONE (t=0.04s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.406
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.667
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.463
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.475
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.519
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.472
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.348
 Average Recall     (AR) @[ IoU=0.50:0.95 

{'AP': 0.4062096,
 'AP50': 0.66650885,
 'AP75': 0.46298265,
 'AP_/Traffic-light': 0.18250729,
 'AP_/crossing-line': 0.6299119,
 'APl': 0.47205508,
 'APm': 0.51882493,
 'APs': 0.47515997,
 'ARl': 0.5986111,
 'ARm': 0.5917647,
 'ARmax1': 0.3478689,
 'ARmax10': 0.46874377,
 'ARmax100': 0.4805957,
 'ARs': 0.5168394}

# convert to tflite format metadata

In [13]:
!pip install tf-nightly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 581.4 MB 25 kB/s 
     |████████████████████████████████| 5.9 MB 58.2 MB/s 
     |████████████████████████████████| 1.7 MB 17.3 MB/s 
     |████████████████████████████████| 439 kB 68.9 MB/s 
  Attempting uninstall: tf-estimator-nightly
    Found existing installation: tf-estimator-nightly 2.8.0.dev2021122109
    Uninstalling tf-estimator-nightly-2.8.0.dev2021122109:
      Successfully uninstalled tf-estimator-nightly-2.8.0.dev2021122109
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed.

In [1]:
!saved_model_cli show --dir /content/ --tag_set serve --all

2022-08-07 12:04:13.643643: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-07 12:04:13.983814: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-08-07 12:04:14.980440: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-08-07 12:04:14.981305: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer

In [2]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model('/content/saved_model',signature_keys=['serving_default'])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.experimental_new_converter = True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,  
                                        tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()

with tf.io.gfile.GFile('/content/saved_model/detect.tflite', 'wb') as f:
  f.write(tflite_model)

# convert to metadata

In [3]:
pip install tflite_support_nightly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 34.8 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tflite-model-maker 0.4.1 requires flatbuffers==1.12, but you have flatbuffers 2.0 which is incompatible.
tf-nightly 2.11.0.dev20220807 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.1 which is incompatible.
tensorflowjs 3.19.0 requires protobuf==3.20.0, but you have protobuf 3.20.1 which is incompatible.
tensorflow 2.8.0+zzzcolab20220506162203 requires tf-estimator-nightly==2.8.0.dev2021122109, but you have tf-estimator-nightly 2.11.0.dev2022080708 which is incompatible.
tb-nightly 2.10.0

In [1]:
%cd /content/drive/MyDrive/data2
!mkdir tflite_with_metadata
%cd ..

/content/drive/MyDrive/data2
mkdir: cannot create directory ‘tflite_with_metadata’: File exists
/content/drive/MyDrive


In [2]:
%cd /content/drive/MyDrive/data2

# Attach Metadata to TFLite
from tflite_support.metadata_writers import object_detector
from tflite_support.metadata_writers import writer_utils
from tflite_support import metadata
import flatbuffers
import os
from tensorflow_lite_support.metadata import metadata_schema_py_generated as _metadata_fb
from tensorflow_lite_support.metadata.python import metadata as _metadata
from tensorflow_lite_support.metadata.python.metadata_writers import metadata_info
from tensorflow_lite_support.metadata.python.metadata_writers import metadata_writer
from tensorflow_lite_support.metadata.python.metadata_writers import writer_utils

ObjectDetectorWriter = object_detector.MetadataWriter

_MODEL_PATH = "/content/saved_model/detect.tflite"
_LABEL_FILE = "/content/labels.txt"
_SAVE_TO_PATH = "/content/drive/MyDrive/data2/tflite_with_metadata/detect.tflite"

writer = ObjectDetectorWriter.create_for_inference(
    writer_utils.load_file(_MODEL_PATH), [127.5], [127.5], [_LABEL_FILE])
writer_utils.save_file(writer.populate(), _SAVE_TO_PATH)

# Verify the populated metadata and associated files.
displayer = metadata.MetadataDisplayer.with_model_file(_SAVE_TO_PATH)
print("Metadata populated:")
print(displayer.get_metadata_json())
print("Associated file(s) populated:")
print(displayer.get_packed_associated_file_list())

model_meta = _metadata_fb.ModelMetadataT()
model_meta.name = "SSD_Detector"
model_meta.description = (
    "Identify which of a known set of objects might be present and provide "
    "information about their positions within the given image or a video "
    "stream.")

# Creates input info.
input_meta = _metadata_fb.TensorMetadataT()
input_meta.name = "image"
input_meta.content = _metadata_fb.ContentT()
input_meta.content.contentProperties = _metadata_fb.ImagePropertiesT()
input_meta.content.contentProperties.colorSpace = (
    _metadata_fb.ColorSpaceType.RGB)
input_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.ImageProperties)
input_normalization = _metadata_fb.ProcessUnitT()
input_normalization.optionsType = (
    _metadata_fb.ProcessUnitOptions.NormalizationOptions)
input_normalization.options = _metadata_fb.NormalizationOptionsT()
input_normalization.options.mean = [127.5]
input_normalization.options.std = [127.5]
input_meta.processUnits = [input_normalization]
input_stats = _metadata_fb.StatsT()
input_stats.max = [255]
input_stats.min = [0]
input_meta.stats = input_stats

# Creates outputs info.
output_location_meta = _metadata_fb.TensorMetadataT()
output_location_meta.name = "location"
output_location_meta.description = "The locations of the detected boxes."
output_location_meta.content = _metadata_fb.ContentT()
output_location_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.BoundingBoxProperties)
output_location_meta.content.contentProperties = (
    _metadata_fb.BoundingBoxPropertiesT())
output_location_meta.content.contentProperties.index = [1, 0, 3, 2]
output_location_meta.content.contentProperties.type = (
    _metadata_fb.BoundingBoxType.BOUNDARIES)
output_location_meta.content.contentProperties.coordinateType = (
    _metadata_fb.CoordinateType.RATIO)
output_location_meta.content.range = _metadata_fb.ValueRangeT()
output_location_meta.content.range.min = 2
output_location_meta.content.range.max = 2

output_class_meta = _metadata_fb.TensorMetadataT()
output_class_meta.name = "category"
output_class_meta.description = "The categories of the detected boxes."
output_class_meta.content = _metadata_fb.ContentT()
output_class_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)
output_class_meta.content.contentProperties = (
    _metadata_fb.FeaturePropertiesT())
output_class_meta.content.range = _metadata_fb.ValueRangeT()
output_class_meta.content.range.min = 2
output_class_meta.content.range.max = 2
label_file = _metadata_fb.AssociatedFileT()
label_file.name = os.path.basename("labelmap.txt")
label_file.description = "Label of objects that this model can recognize."
label_file.type = _metadata_fb.AssociatedFileType.TENSOR_VALUE_LABELS
output_class_meta.associatedFiles = [label_file]

output_score_meta = _metadata_fb.TensorMetadataT()
output_score_meta.name = "score"
output_score_meta.description = "The scores of the detected boxes."
output_score_meta.content = _metadata_fb.ContentT()
output_score_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)
output_score_meta.content.contentProperties = (
    _metadata_fb.FeaturePropertiesT())
output_score_meta.content.range = _metadata_fb.ValueRangeT()
output_score_meta.content.range.min = 2
output_score_meta.content.range.max = 2

output_number_meta = _metadata_fb.TensorMetadataT()
output_number_meta.name = "number of detections"
output_number_meta.description = "The number of the detected boxes."
output_number_meta.content = _metadata_fb.ContentT()
output_number_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)
output_number_meta.content.contentProperties = (
    _metadata_fb.FeaturePropertiesT())

# Creates subgraph info.
group = _metadata_fb.TensorGroupT()
group.name = "detection result"
group.tensorNames = [
    output_location_meta.name, output_class_meta.name,
    output_score_meta.name
]
subgraph = _metadata_fb.SubGraphMetadataT()
subgraph.inputTensorMetadata = [input_meta]
subgraph.outputTensorMetadata = [
    output_location_meta, output_class_meta, output_score_meta,
    output_number_meta
]
subgraph.outputTensorGroups = [group]
model_meta.subgraphMetadata = [subgraph]

b = flatbuffers.Builder(0)
b.Finish(
    model_meta.Pack(b),
    _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
metadata_buf = b.Output()

/content/drive/MyDrive/data2
Metadata populated:
{
  "name": "ObjectDetector",
  "description": "Identify which of a known set of objects might be present and provide information about their positions within the given image or a video stream.",
  "subgraph_metadata": [
    {
      "input_tensor_metadata": [
        {
          "name": "image",
          "description": "Input image to be detected.",
          "content": {
            "content_properties_type": "ImageProperties",
            "content_properties": {
              "color_space": "RGB"
            }
          },
          "process_units": [
            {
              "options_type": "NormalizationOptions",
              "options": {
                "mean": [
                  127.5
                ],
                "std": [
                  127.5
                ]
              }
            }
          ],
          "stats": {
            "max": [
              255.0
            ],
            "min": [
              0.0


In [ ]:
%cd /content/
#!pip install tensorflow-object-detection-api
#Loading the saved_model
import tensorflow as tf
import time
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from PIL import Image
from google.colab.patches import cv2_imshow
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

IMAGE_SIZE = (12, 8) # Output display size as you want
import matplotlib.pyplot as plt
PATH_TO_SAVED_MODEL="/content/"
print('Loading model...', end='')

# Load saved model and build the detection function
detect_fn=tf.saved_model.load(PATH_TO_SAVED_MODEL)
print('Done!')

#Loading the label_map
category_index=label_map_util.create_category_index_from_labelmap("/content/label_map.pbtxt",use_display_name=True)
#category_index=label_map_util.create_category_index_from_labelmap([path_to_label_map],use_display_name=True)

def load_image_into_numpy_array(path):

    return np.array(Image.open(path))

image_path = "/content/abbey-2293953_1920.jpg"
#print('Running inference for {}... '.format(image_path), end='')

image_np = load_image_into_numpy_array(image_path)

# The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
input_tensor = tf.convert_to_tensor(image_np)
# The model expects a batch of images, so add an axis with `tf.newaxis`.
input_tensor = input_tensor[tf.newaxis, ...]

detections = detect_fn(input_tensor)

# All outputs are batches tensors.
# Convert to numpy arrays, and take index [0] to remove the batch dimension.
# We're only interested in the first num_detections.
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.4, # Adjust this value to set the minimum probability boxes to be classified as True
      agnostic_mode=False)
%matplotlib inline
plt.figure(figsize=IMAGE_SIZE, dpi=200)
plt.axis("off")
plt.imshow(image_np_with_detections)
plt.show()